## Prediction
### Deploy ML model using KServe inference service

In [ ]:
import yaml
import json

def deploy_kserve_model(json_file_path):
    try:
        # Run kubectl apply command using subprocess
        subprocess.run(['kubectl', 'apply', '-f', json_file_path], check=True)
        print("Deployment successful!")
    except subprocess.CalledProcessError as e:
        print(f"Deployment failed. Error: {e}")
        

## update kserve-minio-secret
with open('kserve/kserve-minio-secret.yaml', 'r') as file:
    sa_yaml_data = yaml.safe_load(file)

# Convert YAML to JSON
sa_json_data = json.dumps(sa_yaml_data)
sa_data = json.loads(sa_json_data)

sa_data["metadata"]["namespace"] = os.environ['USER']
sa_data['secrets'][0]['name'] = os.environ['USER'] + '-objectstore-secret'

# Save the JSON data to a file
with open('kserve/kserve-minio-secret.json', 'w') as file:
    file.write(json.dumps(sa_data))

# Specify the path to your JSON file
json_file_path = 'kserve/kserve-minio-secret.json'

# Call the function to deploy the Kubernetes resource to kserve service account
deploy_kserve_model(json_file_path)

###################################################################################
## update kserve-mlflow-deploy 
with open('kserve/kserve-mlflow-deploy.yaml', 'r') as file:
    yaml_data = yaml.safe_load(file)

model_name = 'bike-sharing-exp'
with open('best-model-uri.txt','r') as f:
    storage_uri = f.read()
# Convert YAML to JSON
json_data = json.dumps(yaml_data)
data = json.loads(json_data)

data["metadata"]["namespace"] = os.environ['USER']
data["metadata"]["name"] = model_name
data['spec']['predictor']['sklearn']['storageUri'] = storage_uri

# Save the JSON data to a file
with open('kserve/kserve-mlflow-deploy.json', 'w') as file:
    file.write(json.dumps(data))

# Specify the path to your JSON file
json_file_path = 'kserve/kserve-mlflow-deploy.json'

# Call the function to deploy the Kubernetes resource to deploy ml model 
deploy_kserve_model(json_file_path)

### Deployment of kserve will take few minutes to comes to ready state, so wait for few minutes before actually executing the below cell.

In [ ]:
import os
import requests

DOMAIN_NAME = "svc.cluster.local" # change this to your domain for external access 
NAMESPACE = os.environ['USER']
DEPLOYMENT_NAME = model_name
MODEL_NAME = DEPLOYMENT_NAME
SVC = f'{DEPLOYMENT_NAME}-predictor-default.{NAMESPACE}.{DOMAIN_NAME}'
URL = f"https://{SVC}/v2/models/{MODEL_NAME}/infer"

print(URL)

names = ['season', 'year', 'month', 'hour_of_day', 'is_holiday', 'weekday', 'is_workingday', 
         'weather_situation', 'temperature', 'feels_like_temperature', 'humidity', 'windspeed']

input_data = [
    [1, 2, 1, 0, 0, 6, 0, 1, 0.24, 0.2879, 0.81, 0.0000],
    [1, 5, 1, 0, 0, 6, 1, 1, 0.24, 0.2879, 0.81, 0.0000]
]

inputs = {
  "inputs": [
    {
      "name": "ndarray",
      "shape": [2, 12],
      "datatype": "FP32",
      "data": input_data
    }
  ]
}

headers = {"Authorization": f"Bearer {os.environ['AUTH_TOKEN']}"}

response = requests.post(URL, json=inputs, headers=headers, verify=False)

print(response.reason)

output = response.json()['outputs'][0]['data']

print("Rendted Bikes Per Hours:\n")
for item, out in zip(input_data, output):
    input_dict = dict(zip(names,item))
    print(f"Input Data: {input_dict} \n\nBike Per Hour: {out}\n")